In [ ]:
import pandas as pd
import numpy as np
import random
import glob
import cufflinks as cf
import os
import sys
import json

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import cufflinks as cf
cf.go_offline()

from nodescontrollerfolder.nodescontroller import initialize_controller_parameters_nodes, initialize_nodes_conrtol_central, run_nodes_central, run_nodes_control, initialize_volt_control, run_volt_control
from nodescontrollerfolder.feederfunctions import LinearApproxFeeder,DER_coeff_computation,magnitudeDerivative
from opendsssimulator import gridparameters
from opendssdirect import dss

# sys.path.append("/Users/dvaidhyn/work/fy22/aumc/behavior_dev/behavior")
from grid_resilience import *
import mesa
import grid_resilience.model as gm

In [ ]:
sdf = pd.read_csv('data/Solar_data_37.92_-122.29.csv')
sdf.index = pd.date_range(start='2013-04-07 00:00:00', end='2013-04-07 23:59:00', freq = '300S')
ssdf = sdf.resample('1S').bfill()
solarval =  ssdf.dni.values

In [ ]:
ldf = pd.read_csv("data/LoadShape1.CSV",header=None)
edf = pd.read_csv("data/evse_profile.csv",header=0)

ldfval  = ldf[0].values.tolist()
edfval = edf['ev_available'].values.tolist()


In [ ]:
####### simulation start and stop times ##########
starttime = "2013-04-07 16:00:00"
stoptime = "2013-04-07 19:00:00"

In [ ]:
example_opendssfeeder="./feederdata/p1uhs0_1247--p1udt104"

In [ ]:
### Things that do not change. 
################## initialize grid parameters based on Feeder ################
gp = gridparameters(example_opendssfeeder, ".")

# detailed information can be found in the OpenDSS feeder folder
op = "./feederdata/p1uhs0_1247--p1udt104/" # .format("/Users/dvaidhyn/work/fy22/aumc/rbc/feederdata")

example_opendssfeeder_master = f"{example_opendssfeeder}/Master_XZ.dss"
feeder_param = LinearApproxFeeder(example_opendssfeeder_master)
cpl = "Compile {}".format(example_opendssfeeder_master)

dss.run_command(cpl)

In [ ]:
def setup_feeder_ders(gp, feeder_param):
    Nodes_monitor_V = [gp.loadname_mapping_dict[item]
                       ['Node']+1 for item in gp.loadname_mapping_dict.keys()]
    Nodes_monitor_V_index = []
    for pp in range(0, len(feeder_param['Seq_numbers'])):
        if (np.floor(feeder_param['Seq_numbers'][pp]) in Nodes_monitor_V):
            if Nodes_monitor_V_index == []:
                Nodes_monitor_V_index = [pp]
                Nodes_monitor_V_index = np.array(Nodes_monitor_V_index).T
            else:
                Nodes_monitor_V_index = np.column_stack(
                    [Nodes_monitor_V_index.T, pp]).T
    Nodes_monitor_V_index_3 = np.subtract(Nodes_monitor_V_index, 3)
    r, c = Nodes_monitor_V_index_3.shape
    Nodes_monitor_V_index_3 = Nodes_monitor_V_index_3.reshape(r*c)
    nodes_coeff = {}
    connections = ['delta', 'wye']
    phases = [['1'], ['2'], ['3'], ['1', '3'], [
        '1', '2'], ['2', '3'], ['1', '2', '3']]
    parameter_dict = {}
    colls = []
    for x in range(0, feeder_param['Nnode']):

        for y in connections:
            for z in phases:
                phasefeeder_param = '_'.join(z)
                phaselist = list(map(int, z))
                DEF_coeff = DER_coeff_computation(x, y, phaselist, feeder_param['coeffMagWye'], feeder_param['coeffMagDelta'],
                                                  feeder_param['coeffWyeP0'], feeder_param['coeffDeltaP0'], feeder_param['Seq_numbers'], feeder_param)
                if(DEF_coeff == None):
                    pass
                else:
                    A = np.array(DEF_coeff['A'])
                    M = np.array(DEF_coeff['M'])
                    power_index = DEF_coeff['power_index']
                    nodes_coeff['{}_{}_{}'.format(x, y, phasefeeder_param)] = {'Ap': A[Nodes_monitor_V_index_3, 0].real,
                                                                      'Aq': A[Nodes_monitor_V_index_3, 1].real,                                                                'Mp': M[:, 0].real, 'Mq': M[:, 1].real}
                    colls.append(str('{}_{}_{}'.format(x, y, phasefeeder_param)))
    return ({'Nodes_monitor_V_index_3': Nodes_monitor_V_index_3, 'nodes_coeff' : nodes_coeff})

In [ ]:
tmp = setup_feeder_ders(gp, feeder_param)
Nodes_monitor_V_index_3 = tmp['Nodes_monitor_V_index_3']
nodes_coeff = tmp['nodes_coeff']

starttime = 20
stoptime = 1000

default_config = {
        ## Upper and lower voltage limits in p.u.
        'Vupper' : 1.05,
        'Vlower' : 0.95,
        ## simulation start and stop time
        'starttime' :starttime,
        'stoptime' : stoptime,
        }

controller_parameters = {
    ### changes to controller for KP
        'Vupper' : 1.05,
        'Vlower' : 0.95,
        ## add anything your controller needs here. 
        'behavep1' : 1.2,
        "uuid" : 1.0,
        "type" : "rbc",
        "nu" : 0.001, 
        "alpha" : 0.1, 
        "costp" : 1.0,
        "costq" : 1.0,
        "Aq" : [1.0],
        "Ap" : [1.0],
        "Mq" : [1.0],
        "Mp" : [1.0],
        "Nodes_monitor_V_index_3" : [1.0]
}

agent_node_map = {}
############# get grid parameters for this load connection ############
# gp_config = gp.gridparameters(loadname)
# loadname = "TR(R:P1UDT454-P1UDT454LV)".lower()
lds = gp.AllLoadNames
control_logic = "something"

### This loop assigns DERs to the nodes on the feeder

In [ ]:
agent_file = {
    "name": "feeder_0_opendss_primal",
    "feeder_num": 0,
    "helics": {
        "deltat": 10,
        "subscription_topic": [
            "feeder_0_gammauk",
            "feeder_0_gammalk",
            "feeder_0_muk",
            "feeder_0_lambdak",
            "feeder_0_alpha",
            "feeder_0_service"
        ],
        "publication_topic": [
            "feeder_0_primal_nodename1"
        ],
        "endpoints": [
            "feeder_0_opendss_other_primal"
        ]
    },
    "dertype": "",
    "model_type": "",
    "type": "",
    "opendss_resoution": 10,
    "publication_interval": 1,
    "endpoint_interval": 20,
#     "dertype": "Gen",
    "dermodel": "PV",
    "der_profile": None,
    "LoadName": "TR(R:P1UDT471-P1UDT471LV)",
    "model_parameters": {
        "pmax": 71.2,
        "qmax": 14.4,
        "pmin": 0.0,
        "qmin": 0.0,
        "capacity": 71.2,
        "dertype": "Gen",
        "type": "rbc"
    },
    "control_parameters": {
        "Vupper": 1.05,
        "Vlower": 0.95,
        "uuid": 100.33484681672792,
        "uuid": 34343,
        "behavep1": 1.2,
        "type": "rbc",
        "qmax": 14.4,
        "pmin": 0.0,
        "qmin": 0.0,
        "capacity": 71.2,
        "epsilon": 100.0,
        "nu": 100.0,
        "alpha": 5000.0,
        "costp": [
            1.0
        ],
        "costq": [
            1.0
        ],
        "Aq": [
            1.0
        ],
        "Ap": [
            1.0
        ],
        "control_logic": "something",
        "model_type": "PV",
#         "type": "rbc",
        "node": "P1UDT471",
        "loadname": "tr(r:p1udt471-p1udt471lv)",
        "conn": "wye",
        "Node": 11,
        "phase": [
            2
        ],
        "bus": "P1UDT471",
        "voltage_index": 34,
        "sbase": 51833633.33333334,
        "pmax": 71.2,
        "qmax": 14.4,
        "Vupper": 1.05,
        "uuid": 34343,
        "Vlower": 0.95,
        "starttime": "2013-04-07 16:00:00",
        "stoptime": "2013-04-07 19:00:00",
        "Mq": [
            1.0
        ],
        "Mp": [
            1.0
        ],
        "Nodes_monitor_V_index_3": [
            3,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16,
            17,
            18,
            19,
            20,
            21,
            22,
            23,
            24,
            25,
            26,
            27,
            28,
            29,
            30,
            63,
            64
        ]
    },
    "control_logic": "something",
    "model_type": "PV",
    "type": "rbc",
    "node": "P1UDT471",
    "loadname": "tr(r:p1udt471-p1udt471lv)",
    "conn": "wye",
    "Node": 11,
    "phase": [
        2
    ],
    "bus": "P1UDT471",
    "voltage_index": 34,
    "sbase": 51833633.33333334,
    "pmax": 71.2,
    "qmax": 14.4,
    "Vupper": 1.05,
    "uuid": 34343,
    "Vlower": 0.95,
    "Agent": "primalupdate"
}


In [ ]:
agent_file['control_parameters'].keys()

In [ ]:
feeder_num = 0
# loadname = 'TR(R:P1UDT471-P1UDT471LV)'
deltat = 1

control_logic = "controller_type_name"
nodes_subscriptions = ['gammauk',
  'gammalk',
  'muk',
  'lambdak',
  'alpha',
  'service']

In [ ]:

all_esl = []
idx = -1 
for currentload in gp.loadname_mapping_dict.keys():
    idx += 1
    loadnameupper = currentload.upper()
    print(f"Processing load : {loadnameupper}")
    ############# get grid parameters for this load connection ############
    loadname = loadnameupper.lower()
    gp_config = gp.gridparameters(loadname)
    nodes_params = DER_coeff_computation(gp_config['Node'],gp_config['conn'],[lt for lt in gp_config['phase']],feeder_param['coeffMagWye'], feeder_param['coeffMagDelta'],\
                                        feeder_param['coeffWyeP0'], feeder_param['coeffDeltaP0'],\
                                         feeder_param['Seq_numbers'], feeder_param)
    A = nodes_params['A']
    M = nodes_params['M']
    power_index = nodes_params['power_index']
    controller_parameters.update({'Ap': A[Nodes_monitor_V_index_3, 0].real.tolist(),\
                                  'Aq': A[Nodes_monitor_V_index_3, 1].real.tolist(),\
                                  'Mp': M[:, 0].real.tolist(), 'Mq': M[:, 1].real.tolist(),\
                                 'Nodes_monitor_V_index_3': Nodes_monitor_V_index_3.tolist(), 'capacity' : gp_config['pmax'],"model_type" : "PV", 
                                 "volt_index" : gp_config['voltage_index'],
                                'pmax' : gp_config['pmax']*3.3,
                                'qmax' : gp_config['qmax']*3.3,
                                  "costp" : 1.0,
                                  "costq" : 1.0,
                                  "nu" : 10.0,
                                  "alpha": 100.0,
                                  "Vupper" : 1.05, 
                                  "Vlower" : 0.95,
                                  "capacity" : gp_config['pmax']*3.3,

                                'pmin' : 0.0,
                                'qmin' : 0.0,})


    ###### HELICS configuration ######
    helics_params = {
        'subscription_topic' : [f"feeder_{feeder_num}_{x}" for x in nodes_subscriptions],
        'publication_topic' : [f"feeder_{feeder_num}_{loadname}_nodes"],
        'endpoints' : [f"feeder_{feeder_num}_{loadname}_nodes_endpoint"],
        'publication_interval' : 2,
        'endpoint_interval' : 2  , 
        'deltat' : deltat
    }
    model_parameters = {}

    config_init = {
            'der_profile' : None,
            "name" : f"feeder_{feeder_num}_{loadname}_nodes", 
            "LoadName" : loadname.upper(),
            "model_parameters" : model_parameters, 
            "control_parameters" : controller_parameters,
            "control_logic" : control_logic,
            "model_type" : "PV",
            "type" : "rbc",
            "helics" : helics_params,
            "Agent": "primalupdate",
            'publication_interval' : 2,
            'endpoint_interval' : 2  , 
            'deltat' : deltat
            }
    config_init.update(gp_config)
    config_init.update(default_config)
    fname = f"agent{idx}.json"
    f = open(fname, "w")
    json.dump(config_init, f)
    f.close()

############# solar expert conteroller paramerets ############
######### create an SE central controller #########

In [ ]:
!ls


In [ ]:
len(dss.Circuit.AllNodeNames())

In [ ]:
# !mv agent*.json /Users/dvaidhyn/work/fy22/abm/cosim/example/agentdir/.